In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tokenizers.processors import TemplateProcessing
import torch

#tokenizer = AutoTokenizer.from_pretrained('chunwoolee0/circulus-kobart-en-to-ko')
#model = AutoModelForSeq2SeqLM.from_pretrained('/home/hgjeong/hdd1/Project/Capstone/test/model-31epoch-928steps-0.0001loss')  # 여기서 모델 경로를 직접 지정한 경로 대신 원래의 pko-t5-large로 변경했습니다.
#tokenizer.push_to_hub("giliit/capstone_tokenizer")
#model.push_to_hub("giliit/capstone_v2")


tokenizer = AutoTokenizer.from_pretrained('giliit/capstone_tokenizer')
model = AutoModelForSeq2SeqLM.from_pretrained('giliit/capstone_v2')

if not tokenizer.cls_token:
        tokenizer.cls_token = tokenizer.eos_token
if not tokenizer.sep_token:
    tokenizer.sep_token = tokenizer.eos_token

tokenizer._tokenizer.post_processor = TemplateProcessing(
    single=f"{tokenizer.cls_token} $0 {tokenizer.sep_token}",
    pair=f"{tokenizer.cls_token} $A {tokenizer.sep_token} $B:1 {tokenizer.sep_token}:1",
    special_tokens=[(tokenizer.cls_token, tokenizer.cls_token_id), (tokenizer.sep_token, tokenizer.sep_token_id)],
)

In [ ]:
def generate(text):
    processed = {}
    inp = f'{input_text}'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 입력을 처리하기
    tokenizer_input = tokenizer(
        inp,
        padding="max_length",
        max_length=256,
        truncation=True,
        return_tensors='pt'
    )
    
    # 모델과 데이터를 GPU로 이동
    processed["input_ids"] = tokenizer_input["input_ids"].to(device)
    processed["attention_mask"] = tokenizer_input["attention_mask"].to(device)
    model.to(device)
    
    # 모델 실행
    summary_tokens = model.generate(
            processed["input_ids"],
            attention_mask=processed["attention_mask"],
            decoder_start_token_id=tokenizer.bos_token_id,
            max_length=256,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_beams=7,
        )
    
    # 출력 처리
    output = summary_tokens.cpu().detach().tolist()
    print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
input_text = "아이스 아메리카노랑 뜨아 3개씩 줘"

generate(input_text)